In [20]:
import pandas as pd  
import numpy as np  
from sklearn.model_selection import train_test_split  
from sklearn.metrics.pairwise import cosine_similarity  
from sklearn.preprocessing import StandardScaler  

# Load the data  
url = "https://files.grouplens.org/datasets/movielens/ml-100k/u.data"  
columns = ["user_id", "movie_id", "rating", "timestamp"]  
data = pd.read_csv(url, sep='\t', names=columns)  

In [21]:
user_item_matrix = data.pivot_table(  
    index='user_id',   
    columns='movie_id',   
    values='rating',   
    fill_value=0  
)  

In [22]:
class CollaborativeRecommender:  
    def __init__(self, user_item_matrix):  
        self.user_item_matrix = user_item_matrix  
        self.user_similarity = None  
        
    def compute_similarity(self):  
        self.user_similarity = cosine_similarity(self.user_item_matrix)  
        return self.user_similarity  
    
    def recommend_items(self, user_id, n_recommendations=5):
        if user_id not in self.user_item_matrix.index:  
            return "User not found"  
        
        user_index = self.user_item_matrix.index.get_loc(user_id)  
        
        user_ratings = self.user_item_matrix.iloc[user_index]  
        
        similar_users = self.user_similarity[user_index]  
        
        unrated_items = user_ratings[user_ratings == 0].index  
        
        recommendations = {}  
        for item in unrated_items:  
            item_ratings = self.user_item_matrix[item]  
            weighted_ratings = item_ratings * similar_users  
            recommendations[item] = weighted_ratings.sum()  
         
        sorted_recommendations = sorted(  
            recommendations.items(),   
            key=lambda x: x[1],   
            reverse=True  
        )  
        
        return sorted_recommendations[:n_recommendations]  

In [23]:
recommender = CollaborativeRecommender(user_item_matrix)  
recommender.compute_similarity()  

array([[1.        , 0.16693098, 0.04745954, ..., 0.14861694, 0.17950788,
        0.39817474],
       [0.16693098, 1.        , 0.11059132, ..., 0.16148478, 0.17226781,
        0.10579788],
       [0.04745954, 0.11059132, 1.        , ..., 0.10124256, 0.13341615,
        0.02655587],
       ...,
       [0.14861694, 0.16148478, 0.10124256, ..., 1.        , 0.1016418 ,
        0.09511958],
       [0.17950788, 0.17226781, 0.13341615, ..., 0.1016418 , 1.        ,
        0.18246466],
       [0.39817474, 0.10579788, 0.02655587, ..., 0.09511958, 0.18246466,
        1.        ]])

In [24]:
user_id = 196 
recommendations = recommender.recommend_items(user_id)  
print(f"Top recommendations for user {user_id}:")  
for movie, score in recommendations:  
    print(f"Movie ID: {movie}, Recommendation Score: {score}")  

print()

user_id = 11
recommendations = recommender.recommend_items(user_id)  
print(f"Top recommendations for user {user_id}:")  
for movie, score in recommendations:  
    print(f"Movie ID: {movie}, Recommendation Score: {score}")  

Top recommendations for user 196:
Movie ID: 50, Recommendation Score: 277.28806720838395
Movie ID: 100, Recommendation Score: 251.3869923440782
Movie ID: 181, Recommendation Score: 220.12106808739895
Movie ID: 174, Recommendation Score: 204.86558039261453
Movie ID: 127, Recommendation Score: 199.59258099294317

Top recommendations for user 11:
Movie ID: 50, Recommendation Score: 591.6501348556485
Movie ID: 181, Recommendation Score: 484.68496354216427
Movie ID: 174, Recommendation Score: 481.9023167466107
Movie ID: 172, Recommendation Score: 421.42651223764005
Movie ID: 1, Recommendation Score: 416.8942575765386
